# Gera os arquivos para o Embedding Projector das palavras do conjunto de dados SQuAD 2 pt com BERT Transformers by HuggingFace.

Gera os arquivos para Embedding Projector(https://projector.tensorflow.org/).

Pode ser configurado para utilizar o BERTimbau **Large** e **Base**.

Gera arquivos **records_token_sentenca.tsv** com:
- Com e sem pooling dos embeddings das palavras fora do vocabulário.
- Gera embeddings da concatenação das 4 últimas camadas do BERT ou da última camada.

O último registro de records é referente é a média dos embeddings dos tokens do documento consolidados.

O arquivo **meta_token_sentenca.tsv** possui as seguindas colunas:
- Token ou Sentença
- POS-Tag
- OOV (1 - Não existe no vocabulário do **BERT** e combina os *embeddings* dos tokens para formar a palavra e 0 - Existe no vocabulário do **BERT**)
- Id (Id do documento)
- Origem (Id do documento de origem)
- Classe (1 - Original, 0 - Perturbado)
- Perturbada (1 - Perturbada, 0 - Não perturbada)
- Index (Índice da palavra na sentença)
- Próximo token da sentença
- Granularidade (0 - Token, 1 - Sentença)
- Tipo Texto (0 - Palavra perturbada, 1 Palavra Original, 2 - Sentença Perturbada, 3 - Sentença Original)
- Sentença


Exemplo de projeção dos arquivos gerados:
https://projector.tensorflow.org/?config=https://raw.githubusercontent.com/osmarbraz/cohebertv1projecao/main/config_token.json

Repositório dos arquivos no github.
https://github.com/osmarbraz/cohebertv1projecao

---------------------------

Artigos:

- https://arxiv.org/pdf/1611.05469v1.pdf

- https://towardsdatascience.com/visualizing-bias-in-data-using-embedding-projector-649bc65e7487

- https://towardsdatascience.com/bert-visualization-in-embedding-projector-dfe4c9e18ca9

- https://krishansubudhi.github.io/deeplearning/2020/08/27/bert-embeddings-visualization.html

- https://amitness.com/interactive-sentence-embeddings/

---------------------------

**Utiliza o *projeto embeddings* projector para exibir os dados:**
https://projector.tensorflow.org/


**Link biblioteca Huggingface:**
https://github.com/huggingface/transformers


**Artigo original BERT Jacob Devlin:**
https://arxiv.org/pdf/1506.06724.pdf

# 1 Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas
import time
import datetime

#marca o tempo de início do processamento.
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.
    """

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))

    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.

      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.
    """

    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)

    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download
    nome_arquivo = nome_arquivo_destino.split("/")[-1]

    # Define o nome e caminho do arquivo temporário
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"

    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))

    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:
        tamanho_conteudo = data.headers.get("Content-Length")
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):
            if chunk:
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)

    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)

    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """

    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "

    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    sentencas = []
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())

    # Fecha o arquivo
    arquivo.close()

    return sentencas

In [ ]:
def salvar(nome_arquivo,texto):
    """
      Salva um texto em arquivo.

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Classe(ModeloArgumentosMedida) de definição dos parâmetros do modelo para medida

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={'help': 'max seq len'},
    )
    pretrained_model_name_or_path: str = field(
        default='neuralmind/bert-base-portuguese-cased',
        metadata={'help': 'nome do modelo pré-treinado do BERT.'},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={'help': 'define se o texto do modelo deve ser todo em minúsculo.'},
    )
    output_attentions: bool = field(
        default=False,
        metadata={'help': 'habilita se o modelo retorna os pesos de atenção.'},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={'help': 'habilita gerar as camadas ocultas do modelo.'},
    )
    use_wandb : bool = field(
        default=True,
        metadata={'help': 'habilita o uso do wandb.'},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={'help': 'habilita o salvamento do resultado da avaliação.'},
    )
    salvar_medicao : bool = field(
        default=False,
        metadata={'help': 'habilita o salvamento da medicao.'},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={'help': 'habilita o carragamento de mcl ajustado.'},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )
    estrategia_medida: int = field(
        default=0, # 0 - MEAN estratégia média / 1 - MAX  estratégia maior
        metadata={'help': 'Estratégia de cálculo da médida dos embeddings.'},
    )
    equacao_medida: int = field(
        default=0, # 0 - ADJACENTE / 1 - COMBINAÇÃO TODAS / 2 - CONTEXTO
        metadata={'help': 'Equação de cálculo da coerência.'},
    )
    filtro_palavra: int = field(
        default=0, # 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
        metadata={'help': 'Define o filtro de palavras das sentenças para gerar os embeddings.'},
    )

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4  Identificando o ambiente Colab

In [ ]:
# Se estiver executando no Google Colaboratory.
import sys

# Retorna true ou false se estiver no Google Colaboratory.
IN_COLAB = 'google.colab' in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.

      Retorno:
        `device` - Um device de GPU ou CPU.
    """

    # Se existe GPU disponível.
    if torch.cuda.is_available():

        # Diz ao PyTorch para usar GPU.
        device = torch.device("cuda")

        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")

    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Mounted at /content/drive


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 181 kB 67.0 MB/s 
     |████████████████████████████████| 158 kB 52.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 157 kB 39.4 MB/s 
     |████████████████████████████████| 157 kB 45.9 MB/s 
     |████████████████████████████████| 157 kB 42.8 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 157 kB 43.2 MB/s 
     |████████████████████████████████| 157 kB 34.8 MB/s 
     |████████████████████████████████| 157 kB 48.0 MB/s 
     |████████████████████████████████| 156 kB 51.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=75aedf48e1545137eeaf32185782ce9ea425df49adae85ad9c0d7949a6551bbc
  Stored in directory: /root/.cache/pip/wheels/3e/31/

## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 51.5 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
numba 0.56.2 requires setuptools<60, but you have setuptools 65.3.0 which is incompatible.


In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.2
    Uninstalling pydantic-1.9.2:
      Successfully uninstalled pydantic-1.9.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
ERROR: p

## 1.9 Instalação do BERT

Instala a interface pytorch para o BERT by Hugging Face.

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

In [ ]:
!pip install -U transformers==4.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 49.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=7851ec3ef69bbae48ee2a073b88cbe87fe1e8ae388d864a260b6ea3c1a5c99e1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


# 2 Parametrização

## Gerais

In [ ]:
# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 20

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 20

#Realiza o pooling dos tokens de palavras fora do vocabulário do BERT
POOLING_TOKENS = 1 # 0 - Sem pooling / 1 - Com pooling

#Utiliza somente documentos originais, perturbados ou ambos.
CLASSE_DOCUMENTO = 2 # 0 - Somente com a classe 0 (Perturbado) / 1 - Somente com classe 1 (Original), 2 - As duas classes 0 e 1

#Estratégia de recuperação dos embeddings: (1 - Embeddings da última camada,
#                                           2 - Embeddings da concatenação das 4 últimas camadas)
ESTRATEGIA_EMBEDDING = 2

# Estratégias a serem avaliadas (0 - Mean / 1 - Max) para as palavras formadas por mais de um token do BERT
ESTRATEGIA_MEDIDA_STR = ["MEAN", "MAX"]
ESTRATEGIA_MEDIDA = [0, 1]

# Habilita a criação do rótulo "__next__" no projetor para gerar linhas entre os pontos de tokens de uma mesma origem em sequência.
LIGACAO_PROXIMO_TOKEN = True

Permite filtrar os documentos a serem utilizados na geração dos arquivos da projeção.

In [ ]:
IDDO = 19

FILTRO_IDO = ['56cdd21562d2951400fa68b2',   #DO1 Qual foi a intensidade escalada?
             '5a611ac3e9e1cc001a33cf1c',    #DO2 O que substituiu objetos de metal?
             '572679a5f1498d1400e8e100',    #DO3 O que usa um estado de memória interna?
             '5ad4a8225b96ef001a109d20',    #DO4 O que terminou o início da diferenciação de classe?
             '57101890b654c5140001f7d6',    #DO5 Que perguntas correspondem às homossexuais?
             '56d370aa59d6e414001463c2',    #DO6 Onde estava Christina Christian quando foi eliminada do show ?
             '5a8c3316fd22b3001a8d8643',    #DO7 Qual era o nome do padrão incompleto ?
             '56ce34c7aab44d1400b88596',    #DO8 Qual a nacionalidade de Estêvão Gomes ?
             '572ed039c246551400ce46d6',    #DO9 Onde os tornados são mais comuns na Terra ?
             '5a848a6d7cf838001a46a8eb',    #DO10 Qual é o fator chave para a ESA ?
             '5acd38ac07355d001abf3981',    #DO11 O que são neurônios dinâmicos de ampla faixa ?
             '59fc36eaa9fb160018f10dce',    #DO12 Que tipo de mensagem é enviada em um modelo complexo ?
             '56f799d2a6d7ea1400e17260',    #DO13 Quais navios foram atacados em 1852 ?
             '57111fd3b654c5140001fb81',    #DO14 De que cor os cartuchos foram produzidos pela Nintendo para uso interno ?
             '57277da6dd62a815002e9e8b',    #DO15 Qual foi o ato de mediação responsável pela restauração ?
             '5ad261cfd7d075001a429064',    #DO16 Qual é o exemplo mais antigo de linhas de falha financeira ?
             '573121fca5e9cc1400cdbc66',    #DO17 Que tratado trata de armas nucleares ?
             '5ad35760604f3c001a3fddf6',    #DO18 Quem convenceu Michael Dell a usar varejistas para vender PCs ?
             '5acd4d2c07355d001abf3c73',    #DO19 Em que ano Nero foi feito imperador ?
             '5728046cff5b5019007d9b05']    #DO20 Como os bytes de bit são transmitidos ?

FILTRO_STR= ['DO1_','DO2_','DO3_','DO4_','DO5_','DO6_','DO7_','DO8_','DO9_','DO10_','DO11_','DO12_','DO13_','DO14_','DO15_','DO16_','DO17_','DO18_','DO19_','DO20_',]
# FILTRO_STR= ['']

In [ ]:
# Filtra um determinado conjunto de documentos originais e suas versões perturbadas
# FILTRO_DO = [] # Filtro vazio seleciona todos os documentos
FILTRO_DO = [FILTRO_IDO[IDDO]] # Seleciona um documento da lista

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo
model_args = ModeloArgumentosMedida(
    max_seq_len = 512,
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    # pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",
    versao_modelo_spacy = "3.2.0",
    do_lower_case = False,  # default True
    output_attentions = False,  # default False
    output_hidden_states = True, # default False
    use_wandb = True,
    salvar_medicao = True, #Salva o resultado da medição
    salvar_avaliacao = True, # Salva o resultado da avaliação das medições
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara.
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path.
    estrategia_medida = 0, # Atributo usado para os logs do wandb. 0 - MEAN estratégia média / 1 - MAX  estratégia maior
    equacao_medida = 0, # Atributo usado para os logs do wandb. 0 - Palavras adjacentes / 1 - Todas as palavras / 2 - Palavra e contexto
    filtro_palavra = 0 # # Atributo usado para os logs do wandb. 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "SQUAD2_P"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

## Inicialização diretórios

Diretório base local

In [ ]:
# Importando as bibliotecas.
import os

def criaDiretorioLocal():

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):
      # Cria o diretório
      os.makedirs(dirbase)
      logging.info("Diretório criado: {}.".format(dirbase))
  else:
      logging.info("Diretório já existe: {}.".format(dirbase))

In [ ]:
criaDiretorioLocal()

INFO:root:Diretório já existe: /content/SQUAD2_P.


Diretório para conter as os resultados das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedidacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedidacao()

INFO:root:Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/SQUAD2_P/validacao_medicao_palavra.


Diretório para conter os arquivos da avaliação

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioAvaliacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Avaliacao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioAvaliacao()

INFO:root:Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/SQUAD2_P/validacao_medicao_palavra/Avaliacao.


Diretório para conter os arquivos das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedicao():

  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Medicao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedicao()

INFO:root:Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/SQUAD2_P/validacao_medicao_palavra/Medicao.


# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"

    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    """

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy

    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"

    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivo_tar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")
    arquivo_tar.extractall(DIRETORIO_COHEBERT)
    arquivo_tar.close()

    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.

    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.
    """

    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

INFO:root:spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [ ]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).

      Parâmetros:
        `nlp` - Um modelo spaCy carregado.
    """

    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords

Lista dos stopwords

In [ ]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

INFO:root:Quantidade de stopwords: 416.


{'tanta', 'vossa', 'apoio', 'nada', 'seria', 'pelo', 'elas', 'têm', 'quando', 'até', 'bem', 'teus', 'ser', 'porquê', 'além', 'após', 'se', 'sua', 'pode', 'à', 'num', 'questão', 'está', 'baixo', 'primeiro', 'somente', 'nunca', 'isso', 'sétimo', 'primeira', 'certeza', 'aquelas', 'porque', 'teu', 'aí', 'tu', 'depois', 'tentei', 'de', 'só', 'meses', 'favor', 'dezassete', 'pegar', 'na', 'dar', 'ponto', 'vezes', 'pelos', 'como', 'novos', 'for', 'nossos', 'vos', 'desse', 'custa', 'tarde', 'põe', 'nós', 'fazemos', 'local', 'és', 'deverá', 'podia', 'número', 'te', 'ontem', 'estás', 'quê', 'sou', 'naquele', 'ora', 'seus', 'estivemos', 'tive', 'grande', 'põem', 'tal', 'cento', 'partir', 'inclusive', 'nos', 'forma', 'tens', 'aqueles', 'sim', 'boa', 'poderá', 'toda', 'minhas', 'contudo', 'lhe', 'nesse', 'perto', 'menor', 'mil', 'foi', 'dez', 'treze', 'nova', 'parece', 'deve', 'já', 'vocês', 'sobre', 'próxima', 'cá', 'posso', 'segundo', 'faz', 'pelas', 'apenas', 'quero', 'poder', 'dizem', 'sabe', 'p

### getVerbos
Localiza os verbos da sentença

In [ ]:
# Import das bibliotecas.
import spacy
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ]

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ]

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}},
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar
               ]

matcherv = Matcher(nlp.vocab)

matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):
  #Processa o período
  doc1 = nlp(periodo.text)

  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))

  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()

  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [ ]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}

  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [ ]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):

  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))

  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [ ]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [ ]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [ ]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista_tokens = []
  lista_pos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:
    lista_tokens.append(token.text)
    lista_pos.append(token.pos_)

  return lista_tokens, lista_pos

### Tadução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [ ]:
#dicionário que contêm pos tag universal e suas explicações
palavra_universal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}

#Explica a POS
def getPOSPalavraUniversalTraduzido(palavra):
  if palavra in palavra_universal_dict.keys():
      traduzido = palavra_universal_dict[palavra]
  else:
      traduzido = "NA"
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [ ]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [ ]:
def getSentencaSalientePOS(sentenca, pos, tipo_saliente="NOUN"):

  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especeficado
    if pos[i] == tipo_saliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [ ]:
def removeStopWord(documento, stopwords):

  # Remoção das stopwords do documento
  documento_sem_stopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documento_limpo = " ".join(documento_sem_stopwords)

  # Retorna o documento
  return documento_limpo

### retornaRelevante

Retorna somente os palavras do documento ou sentença do tipo especificado.

In [ ]:
def retornaRelevante(documento, classe_relevante="NOUN"):

  # Corrigir!
  # Utilizar o documento já tokenizado pelo spacy!!!!
  # Existe uma lista com o documento e a sentença tokenizada pelo spacy

  # Realiza o parsing no spacy
  doc = nlp(documento)

  # Retorna a lista das palavras relevantes
  documento_com_substantivos = []
  for token in doc:
    #print("token:", token.pos_)
    if token.pos_ == classe_relevante:
      documento_com_substantivos.append(token.text)

  # Concatena o documento com os substantivos
  documento_concatenado = " ".join(documento_com_substantivos)

  # Retorna o documento
  return documento_concatenado

# 4 BERT

## 4.1 Modelo Pré-treinado BERT

### Funções Auxiliares

In [ ]:
def getNomeModeloBERT(model_args):
    '''
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"

    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"

    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"

    return MODELO_BERT

In [ ]:
def getTamanhoBERT(model_args):
    '''
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"

    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"

    return TAMANHO_BERT

### Função download Modelo Pre-treinado BERT

In [ ]:
# Import das bibliotecas.
import zipfile # Biblioteca para descompactar
import shutil # iblioteca de manipulação arquivos de alto nível

def downloadModeloPretreinado(model_args):
    """
      Realiza o download do modelo BERT(MODELO) e retorna o diretório onde o modelo BERT(MODELO) foi descompactado.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `DIRETORIO_MODELO` - Diretório de download do modelo.
    """

    # Nome diretório base modelo BERT
    NOME_DIRETORIO_BASE_MODELO = "modeloBERT"

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Recupera o nome ou caminho do modelo
    MODELO = model_args.pretrained_model_name_or_path

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in MODELO:
        URL_MODELO = MODELO

    # Se a variável foi setada.
    if URL_MODELO:

        # Diretório do modelo.
        DIRETORIO_MODELO = DIRETORIO_COHEBERT + "/" + NOME_DIRETORIO_BASE_MODELO

        # Recupera o nome do arquivo do modelo da url.
        NOME_ARQUIVO = URL_MODELO.split("/")[-1]

        # Nome do arquivo do vocabulário.
        ARQUIVO_VOCAB = "vocab.txt"

        # Caminho do arquivo na url.
        CAMINHO_ARQUIVO = URL_MODELO[0:len(URL_MODELO)-len(NOME_ARQUIVO)]

        # Verifica se o diretório de descompactação existe no diretório corrente
        if os.path.exists(DIRETORIO_MODELO):
            logging.info("Apagando diretório existente do modelo!")
            # Apaga o diretório e os arquivos existentes
            shutil.rmtree(DIRETORIO_MODELO)

        # Realiza o download do arquivo do modelo
        downloadArquivo(URL_MODELO, NOME_ARQUIVO)

        # Descompacta o arquivo no diretório de descompactação.
        arquivo_zip = zipfile.ZipFile(NOME_ARQUIVO, "r")
        arquivo_zip.extractall(DIRETORIO_MODELO)

        # Baixa o arquivo do vocabulário.
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
        URL_MODELO_VOCAB = CAMINHO_ARQUIVO + ARQUIVO_VOCAB
        # Coloca o arquivo do vocabulário no diretório do modelo.
        downloadArquivo(URL_MODELO_VOCAB, DIRETORIO_MODELO + "/" + ARQUIVO_VOCAB)

        # Apaga o arquivo compactado
        os.remove(NOME_ARQUIVO)

        logging.info("Diretório {} do modelo BERT pronta!".format(DIRETORIO_MODELO))

    else:
        DIRETORIO_MODELO = MODELO
        logging.info("Variável URL_MODELO não setada!")

    return DIRETORIO_MODELO

### Copia o modelo do BERT ajustado

In [ ]:
# Import das bibliotecas.
import shutil # iblioteca de manipulação arquivos de alto nível

def copiaModeloAjustado(model_args):
    """
      Copia o modelo ajustado BERT do GoogleDrive para o projeto.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `DIRETORIO_LOCAL_MODELO_AJUSTADO` - Diretório de download ajustado do modelo.
    """

    # Verifica o nome do modelo BERT a ser utilizado
    MODELO_BERT = getNomeModeloBERT(model_args)

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = getTamanhoBERT(model_args)

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Diretório local de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = DIRETORIO_COHEBERT + "/modelo_ajustado/"

    # Diretório remoto de salvamento do modelo no google drive.
    DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/holdout/modelo/" + MODELO_BERT + TAMANHO_BERT

    # Copia o arquivo do modelo para o diretório no Google Drive.
    shutil.copytree(DIRETORIO_REMOTO_MODELO_AJUSTADO, DIRETORIO_LOCAL_MODELO_AJUSTADO)

    logging.info("Modelo BERT ajustado copiado!")

    return DIRETORIO_LOCAL_MODELO_AJUSTADO

### Verifica de onde utilizar o modelo do BERT

In [ ]:
def verificaModelo(model_args):
    """
    Verifica de onde utilizar o modelo.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `DIRETORIO_MODELO` - Diretório de download do modelo.
    """

    DIRETORIO_MODELO = None

    if model_args.usar_mcl_ajustado == True:
        # Diretório do modelo
        DIRETORIO_MODELO = copiaModeloAjustado()

        logging.info("Usando modelo BERT ajustado.")

    else:
        DIRETORIO_MODELO = downloadModeloPretreinado(model_args)
        logging.info("Usando modelo BERT pré-treinado.")

    return DIRETORIO_MODELO

## 4.2 Tokenizador BERT

### Função carrega Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

In [ ]:
# Import das bibliotecas.
from transformers import BertTokenizer # Importando as bibliotecas do tokenizador BERT.

def carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args):
    """
      Carrega o tokenizador do DIRETORIO_MODELO.
      O tokenizador utiliza WordPiece.
      Carregando o tokenizador do diretório "./modelo/" do diretório padrão se variável `DIRETORIO_MODELO` setada.
      Caso contrário carrega da comunidade
      Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.
      O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado a partir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `tokenizer` - Tokenizador BERT.
    """

    tokenizer = None

    # Se a variável DIRETORIO_MODELO foi setada.
    if DIRETORIO_MODELO:
        # Carregando o Tokenizador.
        logging.info("Carregando o tokenizador BERT do diretório {}.".format(DIRETORIO_MODELO))

        tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, do_lower_case=model_args.do_lower_case)

    else:
        # Carregando o Tokenizador da comunidade.
        logging.info("Carregando o tokenizador BERT da comunidade.")

        tokenizer = BertTokenizer.from_pretrained(model_args.pretrained_model_name_or_path, do_lower_case=model_args.do_lower_case)

    return tokenizer

## 4.3 Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

### Função carrega modelo BERT medida

In [ ]:
# Import das bibliotecas.
from transformers import BertModel # Importando as bibliotecas do Modelo BERT.

def carregaModeloMedida(DIRETORIO_MODELO, model_args):
    """
      Carrega o modelo e retorna o modelo.

      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `model` - Um objeto do modelo BERT carregado.
    """

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para cálculo de medidas.".format(DIRETORIO_MODELO))

        model = BertModel.from_pretrained(DIRETORIO_MODELO,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para cálculo de medidas.".format(model_args.pretrained_model_name_or_path))

        model = BertModel.from_pretrained(model_args.pretrained_model_name_or_path,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    return model

### Função carrega o BERT

In [ ]:
def carregaBERT(model_args):
    """
      Carrega o BERT para cálculo de medida ou classificação e retorna o modelo e o tokenizador.
      O tipo do model retornado pode ser BertModel ou BertForSequenceClassification, depende do tipo de model_args.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
          - Se model_args = ModeloArgumentosClassificacao deve ser carregado o BERT para classificação(BertForSequenceClassification).
          - Se model_args = ModeloArgumentosMedida deve ser carregado o BERT para cálculo de medida(BertModel).

      Retorno:
        `model` - Um objeto do modelo BERT carregado.
        `tokenizer` - Um objeto tokenizador BERT carregado.
    """

    # Verifica a origem do modelo
    DIRETORIO_MODELO = verificaModelo(model_args)

    # Variável para conter o modelo
    model = None

    # Carrega o modelo para cálculo da medida
    model = carregaModeloMedida(DIRETORIO_MODELO, model_args)

    # Carrega o tokenizador.
    # O tokenizador é o mesmo para o classificador e medidor.
    tokenizer = carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args)

    return model, tokenizer

### Carrega o BERT

In [ ]:
# Carrega o modelo e tokenizador do BERT
model, tokenizer = carregaBERT(model_args)

INFO:root:Variável URL_MODELO não setada!
INFO:root:Usando modelo BERT pré-treinado.
INFO:root:Carregando o modelo BERT da comunidade neuralmind/bert-large-portuguese-cased para cálculo de medidas.
INFO:root:Carregando o tokenizador BERT do diretório neuralmind/bert-large-portuguese-cased.


### Recupera detalhes do BERT

In [ ]:
# Verifica o nome do modelo BERT a ser utilizado
MODELO_BERT = getNomeModeloBERT(model_args)

# Verifica o tamanho do modelo(default large)
TAMANHO_BERT = getTamanhoBERT(model_args)

## 4.4 Funções auxiliares do BERT

### concatenaListas

In [ ]:
def concatenaListas(lista, pos=1):
  lista_concat = []

  for x in lista:
      lista_concat = lista_concat + x[pos]

  return lista_concat

### getEmbeddingsCamadas

Funções que recuperam os embeddings das camadas:
- Primeira camada;
- Penúltima camada;
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingPrimeiraCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado = output[2][0]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  return resultado

def getEmbeddingPenultimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado = output[2][-2]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  return resultado

def getEmbeddingUltimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado = output[2][-1]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  return resultado

def getEmbeddingSoma4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  embedding_camadas = output[2][-4:]
  # Saída: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  # Usa o método `stack` para criar uma nova dimensão no tensor
  # com a concateção dos tensores dos embeddings.
  #Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado_stack = torch.stack(embedding_camadas, dim=0)
  # Saída: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>

  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultado_stack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>

  return resultado

def getEmbeddingConcat4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Cria uma lista com os tensores a serem concatenados
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)
  # Lista com os tensores a serem concatenados
  lista_concat = []

  # Percorre os 4 últimos
  for i in [-1,-2,-3,-4]:
      # Concatena da lista
      lista_concat.append(output[2][i])

  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)
  # Realiza a concatenação dos embeddings de todos as camadas
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)
  resultado = torch.cat(lista_concat, dim=-1)

  # Saída: Entrada: (<1(lote)> x <qtde_tokens> x <3072 ou 4096>)
  return resultado

def getEmbeddingSomaTodasAsCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas as camadas descontando a primeira(0)
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  embedding_camadas = output[2][1:]
  # Saída: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  # Usa o método `stack` para criar uma nova dimensão no tensor
  # com a concateção dos tensores dos embeddings.
  #Entrada: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado_stack = torch.stack(embedding_camadas, dim=0)
  # Saída: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>

  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultado_stack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>

  return resultado

### getEmbeddingsVisual

Função para gerar as coordenadas de plotagem a partir das sentenças de embeddings.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingsVisualUltimaCamada(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualSoma4UltimasCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualConcat4UltimasCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualSomaTodasAsCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

### getEmbeddings

Função para gerar os embeddings de sentenças.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingsUltimaCamada(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsSoma4UltimasCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsConcat4UltimasCamadas(documento, modelo, tokenizer):
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsSomaTodasAsCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

### getDocumentoTokenizado

Retorna o documento tokenizado

In [ ]:
def getDocumentoTokenizado(documento, tokenizer):
    """
      Retorna o documento tokenizado pelo BERT.

      Parâmetros:
      `documento` - Documento a ser tokenizado.
      `tokenizer` - Tokenizador do BERT.
    """

    # Adiciona os tokens especiais.
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Documento tokenizado
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    del tokenizer

    return documento_tokenizado

### encontrarIndiceSubLista

Retorna os índices de início e fim da sublista na lista

In [ ]:
def encontrarIndiceSubLista(lista: List, sublista: List):
    """
    Localiza os índices de início e fim de uma sublista em uma lista.
    Baseado no algoritmo de https://codereview.stackexchange.com/questions/19627/finding-sub-list
    de  https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

    Parâmetros:
      `lista` - Uma lista.
      `sublista` - Uma sublista a ser localizada na lista.

    Retorno:
      Os índices de início e fim da sublista na lista.
    """
    # Tamanho da lista
    h = len(lista)
    # Tamanho da sblista
    n = len(sublista)
    # Cria um dicionário com os saltos descrescentes dos elementos n-1 da sublista
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    # Percorre a lista
    while i < h:
        # Percorre a sublista
        for j in range(n):
            # Se elemento da lista diferente da sublista pula a interação
            if lista[i - j] != sublista[-j - 1]:
              # Passa para o próximo elemento da lista saltando a sublista
              i += skip.get(lista[i], n)
              # Interrompe o for.
              break
        else:
            #Finalizando a pesquisa depois de executar todo o for(sem break)
            indice_inicio = i - n + 1
            indice_fim = indice_inicio + len(sublista)-1

            # Retorna o início e fim da sublista na lista
            return indice_inicio, indice_fim

    # Não encontrou a sublista na lista
    return -1, -1

### getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras

A partir dos embeddings do documento, localiza o indíce de início e fim de uma sentença no documento e retorna os embeddings da sentença.

In [ ]:
def getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras(embedding_documento,
                                                           token_BERT_documento,
                                                           sentenca,
                                                           tokenizer):

  # Tokeniza a sentença
  sentenca_tokenizada_BERT = getDocumentoTokenizado(sentenca, tokenizer)
  #print(sentenca_tokenizada_BERT)

  # Remove os tokens de início e fim da sentença
  sentenca_tokenizada_BERT.remove("[CLS]")
  sentenca_tokenizada_BERT.remove("[SEP]")
  #print(len(sentenca_tokenizada_BERT))

  # Localiza os índices dos tokens da sentença no documento
  inicio, fim = encontrarIndiceSubLista(token_BERT_documento, sentenca_tokenizada_BERT)
  #print(inicio,fim)

  # Recupera os embeddings dos tokens da sentença a partir dos embeddings do documento
  embedding_sentenca = embedding_documento[inicio:fim+1]
  #print("embedding_sentenca=", embedding_sentenca.shape)

  del tokenizer
  del token_BERT_documento
  del embedding_documento

  # Retorna o embedding da sentença no documento
  return embedding_sentenca, sentenca_tokenizada_BERT

### getEmbeddingDocumentoComTodasPalavrasMean

In [ ]:
# Importa a biblioteca
import torch

def getEmbeddingDocumentoComTodasPalavrasMean(embedding_documento):
  """
    Calcula a média dos embeddings do documento excluindo os tokens
    especiais [CLS] do início e [SEP] do fim.
    Remove primeira dimensão devido ao cálculo da média.

    Parâmetros:
    `embedding_documento` - Embedding do documento.
  """

  # Calcula a média dos embeddings para os tokens de embedding_documento, removendo a primeira dimensão.
  # Entrada: <qtde_tokens> x <768 ou 1024>
  #print("embedding_documento1=", embedding_documento.shape)
  media_embedding_documento = torch.mean(embedding_documento[1:-1], dim=0)
  # Saída: <768 ou 1024>

  del embedding_documento

  return media_embedding_documento

### getEmbeddingDocumentoRelevanteMean

In [ ]:
# Importa a biblioteca
import torch

def getEmbeddingDocumentoRelevanteMean(id_documento,
                                       index_sentenca,
                                       embedding_documento,
                                       token_BERT_documento,
                                       documento,
                                       token_documento,
                                       pos_documento,
                                       filtro):
  """
    Calcula a média dos embeddings do documento considerando tokens do tipo
    especificado no filtro
    Remove primeira dimensão devido ao cálculo da média.

    Parâmetros:
    `embedding_documento` - Embeddings do documento gerados pelo BERT.
    `token_BERT_documento` - Lista com os tokens do documento gerados pelo tokenizador BERT.
    `documento` - Texto com o documento.
    `tokenizer` - Tokenizador do BERT.
    `token_documento` - Lista com os tokens do documento.
    `pos_documento` - Lista com as POS-Tagging do documento.
    `filtro` - Filtro dos embeddings.

  """

  # Recupera a lista de tokens do documento, a lista dos postagging e a lista dos seus embeddings com um mesmo tamanho
  lista_tokens, lista_postagging, lista_embeddings = getTokensEmbeddingsPOSSentenca(id_documento,
                                                                                    index_sentenca,
                                                                                    embedding_documento,
                                                                                    token_BERT_documento,
                                                                                    documento,
                                                                                    token_documento,
                                                                                    pos_documento)

  #print("len(token_BERT_documento):", len(token_BERT_documento))
  #print("token_BERT_documento:", token_BERT_documento)
  #print("len(pos_documento):", len(pos_documento))
  #print("pos_documento:", pos_documento)
  #print("filtro:", filtro)
  #print()

  # Lista com os tensores selecionados
  lista_tokens_selecionados = []
  # Localizar os embeddings dos tokens da sentença tokenizada sem stop word no documento
  for i, token_documento in enumerate(lista_tokens):
      if (lista_postagging[i] in filtro):
          #print("Adicionando palavra do embedding:", lista_tokens[i])
          lista_tokens_selecionados.append(lista_embeddings[i])

  if  len(lista_tokens_selecionados) != 0:
      # Empila os embeddings da lista pela dimensão 0
      embedding_relevante = torch.stack(lista_tokens_selecionados, dim=0)
      #print("embedding_relevante.shape:",embedding_relevante.shape)

      # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
      # Entrada: <qtde_tokens> x <768 ou 1024>
      media_embedding_relevante = torch.mean(embedding_relevante, dim=0)
      # Saída: <768 ou 1024>
      #print("media_embedding_relevante.shape:", media_embedding_relevante.shape)
  else:
      media_embedding_relevante = None

  del embedding_documento
  del token_BERT_documento
  del documento
  del token_documento
  del pos_documento

  return media_embedding_relevante

### getEmbeddingDocumentoMean

Filtros:
- ALL - Sentença com todas as palavras
- NOUN - Sentença somente com substantivos
- VERB - Sentença somente com verbos
- VERB,NOUN - Sentença somente com verbos e substantivos

In [ ]:
def getEmbeddingDocumentoMean(id_documento,
                              index_sentenca,
                              embedding_documento,
                              token_BERT_documento,
                              documento,
                              tokenizer,
                              token_documento,
                              pos_documento,
                              filtro=["ALL"]):
  """
    Rediciona o cálculo da média dos embeddings de acordo com o filtro especificado.

    Parâmetros:
    `embedding_documento` - Embeddings do documento gerados pelo BERT.
    `token_BERT_documento` - Lista com os tokens do documento gerados pelo tokenizador BERT.
    `documento` - Texto com o documento.
    `tokenizer` - Tokenizador do BERT.
    `token_documento` - Lista com os tokens do documento.
    `pos_documento` - Lista com as POS-Tagging do documento.
    `filtro` - Filtro dos embeddings.
  """

  if "ALL" in filtro:
    return getEmbeddingDocumentoComTodasPalavrasMean(embedding_documento)
  else:
    return getEmbeddingDocumentoRelevanteMean(id_documento,
                                              index_sentenca,
                                              embedding_documento,
                                              token_BERT_documento,
                                              documento,
                                              token_documento,
                                              pos_documento,
                                              filtro)

### getTokensEmbeddingsPOSSentenca
Gera os tokens, POS e embeddings de cada sentença.

In [ ]:
# Dicionário de tokens de exceções e seus deslocamentos para considerar mais tokens do BERT em relação ao spaCy
# A tokenização do BERT gera mais tokens que a tokenização das palavras do spaCy
dic_excecao_maior = {"al-Ḥarrānī": 3,
               "al-Battānī": 7,
               "mi.²":3,
               "nm":2,
               "550":2,
               "mg":1,
               "550nm":4,
               "q-glass":5,
               "ômega-6":3,
               str(chr(804)+chr(10217)):1,
               "mm":1,
               "K":1,
               "al-qasim":3,
               "T":1,
               "ḱlew-":2,
               "◌":1,
               "m":1,
               "TGF-β":5,
               "56.º":3,
               "45.º":3,
               "34.º":3,
               "nº":1,
               "200º":1,
               "1º":1,
               "g":1,
               "š":1,
               "ž":1,
               "km²":1,
               "κανὠν":1,
               "7º":1,
               "2º":1,
               "3º":1,
               "19º":1,
               "18º":1,
               "ʱ⟩":1,
               "ð":1,
               "θ":1,
               "Ü-Tsang":4,
               "p˭":1,
               "s˭":1,
               "pʰ":1,
               "ʰp":1,
               "sʰ":1,
               "q":1,
               "Ṣalībī":1,
               "ṣalīb":1,
               "⟨bʰ⟩":3,
               "Ônibus":1,
               "2ª":1,
               "indo-arianas":4,
               "G":1,
               "⟨bd":3,
               "ɡ⟩":2,
               "tʰ":1,}

In [ ]:
def getExcecaoDicMaior(token, dic_excecao_maior):

  valor = dic_excecao_maior.get(token)
  if valor != None:
      return valor
  else:
      return -1

In [ ]:
# Dicionário de tokens de exceções e seus deslocamentos para considerar menos tokens do BERT em relação ao spaCy
# A tokenização do BERT gera menos tokens que a tokenização das palavras do spaCy
dic_excecao_menor = {"1°":1,
                    }

In [ ]:
def getExcecaoDicMenor(token, dic_excecao_menor):

  valor = dic_excecao_menor.get(token)
  if valor != None:
      return valor
  else:
      return -1

Função que retorna os embeddings, tokens e POS da sentença com um mesmo tamanho.

In [ ]:
# Importa a biblioteca
import torch

def getTokensEmbeddingsPOSSentenca(embeddingDocumento,
                                   tokenBERTDocumento,
                                   sentenca):
    """
      Retorna os tokens, as postagging e os embeddings dos tokens igualando a quantidade de tokens do spaCy com a tokenização do BERT de acordo com a estratégia.
      Usa a estratégia MEAN para calcular a média dos embeddings dos tokens que formam uma palavra.
      Usa a estratégia MAX para calcular o valor máximo dos embeddings dos tokens que formam uma palavra.
    """

    #Guarda os tokens e embeddings
    listaTokens = []
    lista_tokens_OOV = []
    lista_embeddings_MEAN = []
    lista_embeddings_MAX = []

    # Gera a tokenização e POS-Tagging da sentença
    sentenca_token, sentenca_pos = getListaTokensPOSSentenca(sentenca)

    # print("\nsentenca          :",sentenca)
    # print("sentenca_token      :",sentenca_token)
    # print("len(sentenca_token) :",len(sentenca_token))
    # print("sentenca_pos        :",sentenca_pos)
    # print("len(sentenca_pos)   :",len(sentenca_pos))

    # Recupera os embeddings da sentença dos embeddings do documento
    embeddingSentenca = embeddingDocumento
    sentenca_tokenizada_BERT = tokenBERTDocumento

    # embedding <qtde_tokens x 4096>
    # print("embeddingSentenca          :",embeddingSentenca.shape)
    # print("sentenca_tokenizada_BERT     :",sentenca_tokenizada_BERT)
    # print("len(sentenca_tokenizada_BERT):",len(sentenca_tokenizada_BERT))

    # Seleciona os pares de palavra a serem avaliadas
    pos_wi = 0 # Posição do token da palavra gerado pelo spaCy
    pos_wj = pos_wi # Posição do token da palavra gerado pelo BERT
    pos2 = -1

    # Enquanto o indíce da palavra pos_wj(2a palavra) não chegou ao final da quantidade de tokens do BERT
    while pos_wj < len(sentenca_tokenizada_BERT):

      # Seleciona os tokens da sentença
      wi = sentenca_token[pos_wi] # Recupera o token da palavra gerado pelo spaCy
      wi1 = ""
      pos2 = -1
      if pos_wi+1 < len(sentenca_token):
        wi1 = sentenca_token[pos_wi+1] # Recupera o próximo token da palavra gerado pelo spaCy

        # Localiza o deslocamento da exceção
        pos2 = getExcecaoDicMenor(wi+wi1, dic_excecao_menor)
        #print("Exceção pos2:", pos2)

      wj = sentenca_tokenizada_BERT[pos_wj] # Recupera o token da palavra gerado pelo BERT
      # print("wi[",pos_wi,"]=", wi)
      # print("wj[",pos_wj,"]=", wj)

      # Tratando exceções
      # Localiza o deslocamento da exceção
      pos = getExcecaoDicMaior(wi, dic_excecao_maior)
      #print("Exceção pos:", pos)

      if pos != -1 or pos2 != -1:
        if pos != -1:
          #print("Adiciona 1 Exceção palavra == wi or palavra = [UNK]:",wi)
          listaTokens.append(wi)
          # Marca como fora do vocabulário do BERT
          lista_tokens_OOV.append(1)
          # Verifica se tem mais de um token
          if pos != 1:
            indice_token = pos_wj + pos
            #print("Calcula a média de :", pos_wj , "até", indice_token)
            embeddings_tokens_palavra = embeddingSentenca[pos_wj:indice_token]
            #print("embeddings_tokens_palavra:",embeddings_tokens_palavra.shape)
            # calcular a média dos embeddings dos tokens do BERT da palavra
            embedding_estrategia_MEAN = torch.mean(embeddings_tokens_palavra, dim=0)
            #print("embedding_estrategia_MEAN:",embedding_estrategia_MEAN.shape)
            lista_embeddings_MEAN.append(embedding_estrategia_MEAN)

            # calcular o máximo dos embeddings dos tokens do BERT da palavra
            embedding_estrategia_MAX, linha = torch.max(embeddings_tokens_palavra, dim=0)
            #print("embedding_estrategia_MAX:",embedding_estrategia_MAX.shape)
            lista_embeddings_MAX.append(embedding_estrategia_MAX)
          else:
            # Adiciona o embedding do token a lista de embeddings
            lista_embeddings_MEAN.append(embeddingSentenca[pos_wj])
            lista_embeddings_MAX.append(embeddingSentenca[pos_wj])

          # Avança para a próxima palavra e token do BERT
          pos_wi = pos_wi + 1
          pos_wj = pos_wj + pos
          #print("Proxima:")
          #print("wi[",pos_wi,"]=", sentenca_token[pos_wi])
          #print("wj[",pos_wj,"]=", sentenca_tokenizada_BERT[pos_wj])
        else:
          if pos2 != -1:
            #print("Adiciona 1 Exceção palavra == wi or palavra = [UNK]:",wi)
            listaTokens.append(wi+wi1)
            # Marca como fora do vocabulário do BERT
            lista_tokens_OOV.append(1)
            # Verifica se tem mais de um token
            if pos2 == 1:
              # Adiciona o embedding do token a lista de embeddings
              lista_embeddings_MEAN.append(embeddingSentenca[pos_wj])
              lista_embeddings_MAX.append(embeddingSentenca[pos_wj])

            # Avança para a próxima palavra e token do BERT
            pos_wi = pos_wi + 2
            pos_wj = pos_wj + pos2
            #print("Proxima:")
            #print("wi[",pos_wi,"]=", sentenca_token[pos_wi])
            #print("wj[",pos_wj,"]=", sentenca_tokenizada_BERT[pos_wj])
      else:
        # Tokens iguais adiciona a lista, o token não possui subtoken
        if (wi == wj or wj=="[UNK]"):
          # Adiciona o token a lista de tokens
          #print("Adiciona 2 wi==wj or wj==[UNK]:", wi )
          listaTokens.append(wi)
          # Marca como dentro do vocabulário do BERT
          lista_tokens_OOV.append(0)
          # Adiciona o embedding do token a lista de embeddings
          lista_embeddings_MEAN.append(embeddingSentenca[pos_wj])
          lista_embeddings_MAX.append(embeddingSentenca[pos_wj])
          #print("embedding1[pos_wj]:", embeddingSentenca[pos_wj].shape)
          # Avança para a próxima palavra e token do BERT
          pos_wi = pos_wi + 1
          pos_wj = pos_wj + 1

        else:
          # A palavra foi tokenizada pelo Wordpice com ## ou diferente do spaCy ou desconhecida
          # Inicializa a palavra a ser montada
          palavraPOS = wj
          indice_token = pos_wj + 1
          while  ((palavraPOS != wi) and indice_token < len(sentenca_tokenizada_BERT)):
              if "##" in sentenca_tokenizada_BERT[indice_token]:
                # Remove os caracteres "##" do token
                parte = sentenca_tokenizada_BERT[indice_token][2:]
              else:
                parte = sentenca_tokenizada_BERT[indice_token]

              palavraPOS = palavraPOS + parte
              #print("palavraPOS:",palavraPOS)
              # Avança para o próximo token do BERT
              indice_token = indice_token + 1

          #print("\nMontei palavra:",palavraPOS)
          if (palavraPOS == wi or palavraPOS == "[UNK]"):
              # Adiciona o token a lista
              #print("Adiciona 3 palavra == wi or palavraPOS = [UNK]:",wi)
              listaTokens.append(wi)
              # Marca como fora do vocabulário do BERT
              lista_tokens_OOV.append(1)
              # Calcula a média dos tokens da palavra
              #print("Calcula o máximo :", pos_wj , "até", indice_token)
              embeddings_tokens_palavra = embeddingSentenca[pos_wj:indice_token]
              #print("embeddings_tokens_palavra2:",embeddings_tokens_palavra)
              #print("embeddings_tokens_palavra2:",embeddings_tokens_palavra.shape)

              # calcular a média dos embeddings dos tokens do BERT da palavra
              embedding_estrategia_MEAN = torch.mean(embeddings_tokens_palavra, dim=0)
              #print("embedding_estrategia_MEAN:",embedding_estrategia_MEAN)
              #print("embedding_estrategia_MEAN.shape:",embedding_estrategia_MEAN.shape)
              lista_embeddings_MEAN.append(embedding_estrategia_MEAN)

              # calcular o valor máximo dos embeddings dos tokens do BERT da palavra
              embedding_estrategia_MAX, linha = torch.max(embeddings_tokens_palavra, dim=0)
              #print("embedding_estrategia_MAX:",embedding_estrategia_MAX)
              #print("embedding_estrategia_MAX.shape:",embedding_estrategia_MAX.shape)
              lista_embeddings_MAX.append(embedding_estrategia_MAX)

          # Avança para o próximo token do spaCy
          pos_wi = pos_wi + 1
          # Pula para o próximo token do BERT
          pos_wj = indice_token

    # Verificação se as listas estão com o mesmo tamanho
    #if (len(listaTokens) != len(sentenca_token)) or (len(lista_embeddings_MEAN) != len(sentenca_token)):
    if (len(listaTokens) !=  len(lista_embeddings_MEAN)):
       print("\nsentenca                :",sentenca)
       print("sentenca_pos            :",sentenca_pos)
       print("sentenca_token          :",sentenca_token)
       print("sentenca_tokenizada_BERT  :",sentenca_tokenizada_BERT)
       print("listaTokens             :",listaTokens)
       print("len(listaTokens)        :",len(listaTokens))
       print("lista_embeddings_MEAN     :",lista_embeddings_MEAN)
       print("len(lista_embeddings_MEAN):",len(lista_embeddings_MEAN))
       print("lista_embeddings_MAX      :",lista_embeddings_MAX)
       print("len(lista_embeddings_MAX) :",len(lista_embeddings_MAX))

    del embeddingSentenca
    del tokenBERTDocumento
    del sentenca_tokenizada_BERT
    del sentenca_token

    return listaTokens, sentenca_pos, lista_tokens_OOV, lista_embeddings_MEAN, lista_embeddings_MAX

# 5 - Exemplo de projeção sem pooling

Apresenta os tokens gerados pelo BERT e seus embeddings.

## 5.1 Carregamento dos arquivos de dados originais e perturbados

### 5.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "originalpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

NOME_ARQUIVO_PERTURBADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_COMPACTADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"
NOME_ARQUIVO_PERTURBADO_POS = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.1.2 Cria o diretório local para receber os dados

In [ ]:
# Import das bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Perturbados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}".format(dirbase))
else:
    logging.info("Diretório já existe: {}".format(dirbase))

INFO:root:Diretório já existe: /content/SQUAD2_P


### 5.1.3 Copia e descompacta os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia!")

INFO:root:Terminei a cópia!


Descompacta os arquivos.

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação!")

INFO:root:Terminei a descompactação!


### 5.1.4 Carregamento das lista com os dados dos arquivos originais e perturbados

#### Carrega o arquivo dos dados originais e POS

In [ ]:
#Biblioteca
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

INFO:root:TERMINADO ORIGINAIS: 1419.
INFO:root:TERMINADO ORIGINAIS POS: 1419.


In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento,respondivel
561,57324e4cb9d445190005ea08,['Quando o Japão se rendeu?'],Quando o Japão se rendeu?,1
536,5733d858d058e614000b63c8,['O que significa o lema de Montana?'],O que significa o lema de Montana?,1
900,5a42d0914a4859001aac733f,['Quais fenômenos assimétricos são comuns na m...,Quais fenômenos assimétricos são comuns na mec...,0
102,56df93a338dc42170015207a,['Quanto Wilber disse que Bell deu a ele para ...,Quanto Wilber disse que Bell deu a ele para ex...,1
25,5726a9f3f1498d1400e8e680,['Quanto da floresta tropical permanece hoje n...,Quanto da floresta tropical permanece hoje na ...,1


In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
320,5acfce2e77cf76001a6860d4,"[[['Quantos', 'trabalhos', 'expostos', 'no', '..."
1232,5a60de02e9e1cc001a33cdb6,"[[['O', 'que', 'causa', 'a', 'falta', 'de', 'u..."
835,5728b8862ca10214002da658,"[[['Que', 'governante', 'se', 'opôs', 'ao', 'u..."
5,5acd38ac07355d001abf3981,"[[['O', 'que', 'são', 'neurônios', 'dinâmicos'..."
824,56cff221234ae51400d9c140,"[[['Qual', 'dos', 'maiores', 'fabricantes', 'd..."


#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

def corrigirTipoDadosColunasOriginais(lista_documentos_originais, lista_documentos_originais_pos):

  # Corrige os tipos dos dados
  tipos = {"id": str}
  lista_documentos_originais = lista_documentos_originais.astype(tipos)
  lista_documentos_originais_pos = lista_documentos_originais_pos.astype(tipos)

  # Verifica se o tipo da coluna não é list e converte
  lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

  lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

  logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
  logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

  return lista_documentos_originais, lista_documentos_originais_pos

In [ ]:
lista_documentos_originais, lista_documentos_originais_pos = corrigirTipoDadosColunasOriginais(lista_documentos_originais, lista_documentos_originais_pos)

INFO:root:TERMINADO CORREÇÃO ORIGINAIS: 1419.
INFO:root:TERMINADO CORREÇÃO ORIGINAIS POS: 1419.


#### Criando dados indexados originais

In [ ]:
# Especifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_originais_indexado = lista_documentos_originais.set_index(["id"])
lista_documentos_originais_indexado.head()

,sentencas,documento,respondivel
id,,,
5a8d89b5df8bba001a0f9afb,[O formulário Edna do Link é mais rápido do qu...,O formulário Edna do Link é mais rápido do que...,0
5acfa4e977cf76001a6856da,[Quais dois ministros lutaram pelo poder em An...,Quais dois ministros lutaram pelo poder em Anne?,0
5ad19f40645df0001a2d213b,[O que Irving Langmuir descobriu que aumentari...,O que Irving Langmuir descobriu que aumentaria...,0
56ce66aeaab44d1400b8875a,[Em que ano a célula solar de silício cristali...,Em que ano a célula solar de silício cristalin...,1
5acdabd307355d001abf48f0,[Desde que ano foi levantada a idéia de um tún...,Desde que ano foi levantada a idéia de um túne...,0


In [ ]:
# Especifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_originais_pos_indexado = lista_documentos_originais_pos.set_index(["id"])
lista_documentos_originais_pos_indexado.head()

,pos_documento
id,
5a8d89b5df8bba001a0f9afb,"[[[O, formulário, Edna, do, Link, é, mais, ráp..."
5acfa4e977cf76001a6856da,"[[[Quais, dois, ministros, lutaram, pelo, pode..."
5ad19f40645df0001a2d213b,"[[[O, que, Irving, Langmuir, descobriu, que, a..."
56ce66aeaab44d1400b8875a,"[[[Em, que, ano, a, célula, solar, de, silício..."
5acdabd307355d001abf48f0,"[[[Desde, que, ano, foi, levantada, a, idéia, ..."


#### Carrega o arquivo dos dados perturbados e POS

In [ ]:
# Abre o arquivo e retorna o DataFrame
lista_documentos_perturbados = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO, sep=";", encoding="UTF-8")
lista_documentos_perturbados_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO PERTURBADOS: {}.".format(len(lista_documentos_perturbados)))
logging.info("TERMINADO PERTURBADOS POS: {}.".format(len(lista_documentos_perturbados_pos)))

INFO:root:TERMINADO PERTURBADOS: 28380.
INFO:root:TERMINADO PERTURBADOS POS: 28380.


Alguns csv estão com os nomes das colunas errados

In [ ]:
lista_documentos_perturbados = lista_documentos_perturbados.rename(columns={'documentoPerturbado': 'documento_perturbado'})

In [ ]:
lista_documentos_perturbados.sample(5)

,id,perturbado,documento_perturbado,sentencas
15271,572635e3271a42140099d78a_pert_11,['Como um campeão quer perder um campeonato ?'],Como um campeão quer perder um campeonato ?,[['Como um campeão [MASK] perder um campeonato...
7649,5726d7ebf1498d1400e8ecd4_pert_9,"['A "" dança do tapa "" provavelmente imita os m...","A "" dança do tapa "" provavelmente imita os mov...","[['A "" dança do tapa "" provavelmente imita os ..."
22258,570c2effec8fbc190045bd52_pert_18,['A que foi útil o Programa Top Hoodlum ?'],A que foi útil o Programa Top Hoodlum ?,"[['A que foi [MASK] o Programa Top Hoodlum ?',..."
20880,572995f63f37b319004784ab_pert_0,['Em que medida é a arquitetura neoclássica ma...,Em que medida é a arquitetura neoclássica mais...,[['Em que [MASK] é a arquitetura neoclássica m...
4169,5aced34532bba1001ae4b612_pert_9,['O que está sendo desenvolvido ?'],O que está sendo desenvolvido ?,"[['O que está sendo [MASK] ?', 'descartado', '..."


In [ ]:
lista_documentos_perturbados_pos.sample(5)

,id,pos_documento
5946,5ad1a38a645df0001a2d2162_pert_6,"[[['Quanta', 'água', 'aumentou', 'significativ..."
13364,5ad27baad7d075001a42965d_pert_4,"[[['Como', 'eles', 'não', 'sabia', 'que', 'seu..."
26443,5729499d1d04691400779253_pert_3,"[[['Qual', 'é', 'o', 'nome', 'do', 'casal', 'd..."
6737,572fa080947a6a140053cae0_pert_17,"[[['Os', 'Estados', 'Unidos', 'e', 'muitos', '..."
27251,56dff2277aa994140058e29c_pert_11,"[[['Que', 'característica', 'notável', 'do', '..."


#### Corrigir os tipos de colunas dos dados perturbados e POS

Em dados perturbados:
- coluna 1 - `perturbado` carregadas do arquivo vem como string e não como lista.
- coluna 3 - `sentencas` carregadas do arquivo vem como string e não como lista.

Em dados perturbados pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

def corrigirTipoDadosColunasPerturbados(lista_documentos_perturbados, lista_documentos_perturbados_pos):

  # Corrige os tipos dos dados
  tipos = {"id": str}
  lista_documentos_perturbados = lista_documentos_perturbados.astype(tipos)
  lista_documentos_perturbados_pos = lista_documentos_perturbados_pos.astype(tipos)

  # Verifica se o tipo da coluna não é list e converte
  lista_documentos_perturbados["perturbado"] = lista_documentos_perturbados["perturbado"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)
  lista_documentos_perturbados["sentencas"] = lista_documentos_perturbados["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

  lista_documentos_perturbados_pos["pos_documento"] = lista_documentos_perturbados_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

  logging.info("TERMINADO CORREÇÃO PERTURBADO: {}.".format(len(lista_documentos_perturbados)))
  logging.info("TERMINADO CORREÇÃO PERTURBADO POS: {}.".format(len(lista_documentos_perturbados_pos)))

  return lista_documentos_perturbados, lista_documentos_perturbados_pos

In [ ]:
 lista_documentos_perturbados, lista_documentos_perturbados_pos = corrigirTipoDadosColunasPerturbados(lista_documentos_perturbados, lista_documentos_perturbados_pos)

INFO:root:TERMINADO CORREÇÃO PERTURBADO: 28380.
INFO:root:TERMINADO CORREÇÃO PERTURBADO POS: 28380.


#### Criando dados indexados perturbados

In [ ]:
# Especifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_perturbados_indexado = lista_documentos_perturbados.set_index(["id"])
lista_documentos_perturbados_indexado.head()

,perturbado,documento_perturbado,sentencas
id,,,
5a8d89b5df8bba001a0f9afb_pert_0,[O form Edna do Link é mais rápido do que outr...,O form Edna do Link é mais rápido do que outro...,[[O [MASK] Edna do Link é mais rápido do que o...
5a8d89b5df8bba001a0f9afb_pert_1,[O Form Edna do Link é mais rápido do que outr...,O Form Edna do Link é mais rápido do que outro...,[[O [MASK] Edna do Link é mais rápido do que o...
5a8d89b5df8bba001a0f9afb_pert_2,[O Can Edna do Link é mais rápido do que outro...,O Can Edna do Link é mais rápido do que outro ...,[[O [MASK] Edna do Link é mais rápido do que o...
5a8d89b5df8bba001a0f9afb_pert_3,[O da Edna do Link é mais rápido do que outro ...,O da Edna do Link é mais rápido do que outro f...,[[O [MASK] Edna do Link é mais rápido do que o...
5a8d89b5df8bba001a0f9afb_pert_4,[O Cada Edna do Link é mais rápido do que outr...,O Cada Edna do Link é mais rápido do que outro...,[[O [MASK] Edna do Link é mais rápido do que o...


In [ ]:
# Especifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_perturbados_pos_indexado = lista_documentos_perturbados_pos.set_index(["id"])
lista_documentos_perturbados_pos_indexado.head()

,pos_documento
id,
5a8d89b5df8bba001a0f9afb_pert_0,"[[[O, form, Edna, do, Link, é, mais, rápido, d..."
5a8d89b5df8bba001a0f9afb_pert_1,"[[[O, Form, Edna, do, Link, é, mais, rápido, d..."
5a8d89b5df8bba001a0f9afb_pert_2,"[[[O, Can, Edna, do, Link, é, mais, rápido, do..."
5a8d89b5df8bba001a0f9afb_pert_3,"[[[O, da, Edna, do, Link, é, mais, rápido, do,..."
5a8d89b5df8bba001a0f9afb_pert_4,"[[[O, Cada, Edna, do, Link, é, mais, rápido, d..."


### 5.1.5 Agrupar os dados originais e perturbados

In [ ]:
# Import das bibliotecas.
import ast
from tqdm.notebook import tqdm as tqdm_notebook

def agruparDadosOriginaisPerturbados(lista_documentos_originais, lista_documentos_perturbados_indexado):

  print("Processando",len(lista_documentos_originais),"documentos originais")

  lista_documentos_agrupados = []

  # Se tem algum id no lista do filtro seleciona os documentos originais
  if len(FILTRO_DO) != 0:
    lista_filtro = lista_documentos_originais[lista_documentos_originais['id'].isin(FILTRO_DO)]

    # Barra de progresso dos documentos
    lista_documentos_originais_bar = tqdm_notebook(lista_filtro.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_filtro))
  else:
    # Barra de progresso dos documentos
    lista_documentos_originais_bar = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_originais))

  # Percorre os documentos
  for i, linha_documento in lista_documentos_originais_bar:
      #if i < 2:
      #print("linha_documento:",linha_documento)
      # Recupera o id do documento
      id_documento = linha_documento[0]

      # Carrega a lista das sentenças do documento
      lista_sentenca_documento = linha_documento[1]
      #print("\nlista_sentenca_documento:",lista_sentenca_documento)
      #print("len(lista_sentenca_documento):",len(lista_sentenca_documento))

      # Adiciona o original a lista dos dados agrupados, considerando como coerente(1)
      lista_documentos_agrupados.append([id_documento, lista_sentenca_documento, linha_documento[2], 1])

      # Percorre os documentos perturbados apartir do original
      for j in range(0, model_args.documentos_perturbados):

        # Id do documento perturbado
        id_perturbado = str(id_documento) + "_pert_" + str(j)

        # localiza o documento perturbado
        #documento_perturbado = lista_documentos_perturbados.loc[lista_documentos_perturbados['id']==id_perturbado].values[0]
        documento_perturbado = lista_documentos_perturbados_indexado.loc[id_perturbado]
        # Recupera a sentença do documento perturbado
        lista_perturbado = documento_perturbado[0]

        # Adiciona o perturbado a lista dos dados agrupados considerando como incoerente(0)
        lista_documentos_agrupados.append([id_perturbado, lista_perturbado, documento_perturbado[1], 0])

  logging.info("TERMINADO AGRUPAMENTO: {}.".format(len(lista_documentos_agrupados)))

  # Cria o dataframe da lista
  lista_documentos_agrupados = pd.DataFrame(lista_documentos_agrupados, columns = ["id","sentencas","documento","classe"])

  # Corrige os tipos dos dados da lista agrupada
  tipos = {"id": str, "sentencas": object, "documento": str, "classe": int}

  lista_documentos_agrupados = lista_documentos_agrupados.astype(tipos)

  return lista_documentos_agrupados

In [ ]:
# Importa das bibliotecas
import pandas as pd

print("Analisando documentos originais e perturbados")
# Concatena as listas de documentos originais e perturbados
lista_documentos_agrupados = agruparDadosOriginaisPerturbados(lista_documentos_originais, lista_documentos_perturbados_indexado)
lista_documentos_agrupados_pos = pd.concat([lista_documentos_originais_pos, lista_documentos_perturbados_pos])

# Corrige o tipo de dado da coluna id da lista
tipos = {"id": str}
lista_documentos_agrupados_pos = lista_documentos_agrupados_pos.astype(tipos)

Analisando documentos originais e perturbados
Processando 1419 documentos originais


Documentos:   0%|          | 0/1 [00:00<?, ? documento/s]

INFO:root:TERMINADO AGRUPAMENTO: 21.


In [ ]:
logging.info("TERMINADO AGRUPAMENTO: {}.".format(len(lista_documentos_agrupados)))

INFO:root:TERMINADO AGRUPAMENTO: 21.


In [ ]:
lista_documentos_agrupados.sample(5)

,id,sentencas,documento,classe
10,5728046cff5b5019007d9b05_pert_9,[Como os bytes de bit são organizados ?],Como os bytes de bit são organizados ?,0
13,5728046cff5b5019007d9b05_pert_12,[Como os bytes de bit são construídos ?],Como os bytes de bit são construídos ?,0
16,5728046cff5b5019007d9b05_pert_15,[Como os bytes de bit são feitos ?],Como os bytes de bit são feitos ?,0
7,5728046cff5b5019007d9b05_pert_6,[Como os bytes de bit são definidos ?],Como os bytes de bit são definidos ?,0
18,5728046cff5b5019007d9b05_pert_17,[Como os bytes de bit são transmitidos ?],Como os bytes de bit são transmitidos ?,0


In [ ]:
logging.info("TERMINADO AGRUPAMENTO POS: {}.".format(len(lista_documentos_agrupados_pos)))

INFO:root:TERMINADO AGRUPAMENTO POS: 29799.


#### Criar dados indexados

In [ ]:
# Especifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_agrupados_indexado = lista_documentos_agrupados.set_index(["id"])
lista_documentos_agrupados_indexado.head()

,sentencas,documento,classe
id,,,
5728046cff5b5019007d9b05,[Como os bytes de bit são transmitidos?],Como os bytes de bit são transmitidos?,1
5728046cff5b5019007d9b05_pert_0,[Como os bytes de bit são classificados ?],Como os bytes de bit são classificados ?,0
5728046cff5b5019007d9b05_pert_1,[Como os bytes de bit são escritos ?],Como os bytes de bit são escritos ?,0
5728046cff5b5019007d9b05_pert_2,[Como os bytes de bit são gravados ?],Como os bytes de bit são gravados ?,0
5728046cff5b5019007d9b05_pert_3,[Como os bytes de bit são formados ?],Como os bytes de bit são formados ?,0


In [ ]:
# Especifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_agrupados_pos_indexado = lista_documentos_agrupados_pos.set_index(["id"])
lista_documentos_agrupados_pos_indexado.head()

,pos_documento
id,
5a8d89b5df8bba001a0f9afb,"[[[O, formulário, Edna, do, Link, é, mais, ráp..."
5acfa4e977cf76001a6856da,"[[[Quais, dois, ministros, lutaram, pelo, pode..."
5ad19f40645df0001a2d213b,"[[[O, que, Irving, Langmuir, descobriu, que, a..."
56ce66aeaab44d1400b8875a,"[[[Em, que, ano, a, célula, solar, de, silício..."
5acdabd307355d001abf48f0,"[[[Desde, que, ano, foi, levantada, a, idéia, ..."


### 5.1.6 Funções auxiliares

#### getIndicePalavraPerturbada

Retorna o índice da palavra perturbada em um documento

In [ ]:
def getIndicePalavraPerturbada(_id_perturbado):

  # print("_id_perturbado:",_id_perturbado)

  # localiza os dados do documento perturbado mascarado
  reg_documento_perturbado = lista_documentos_perturbados_indexado.loc[_id_perturbado]

  # Recupera a lista das sentenças perturbadas
  lista_sentencas_mascarada = reg_documento_perturbado[2]

  # Índice da sentença perturbada
  index_sentenca = -1

  # Percorre as sentenças para encontrar a sentença perturbada
  for i, linha in enumerate(lista_sentencas_mascarada):

    # Identifica a sentença mascarada que foi perturbada
    if 'MASK' in linha[0] :
      # Recupera a palavra mascarada sentença do documento perturbado
      index_sentenca = i
      sentenca_mascarada = linha[0]
      palavra_mascarada = linha[1]
      token_predito = linha[2]
      peso_predito = linha[3]

  # localiza os dados do documento perturbado pos
  reg_documento_perturbado_pos = lista_documentos_perturbados_pos_indexado.loc[_id_perturbado]
  # print("reg_documento_perturbado_pos:",reg_documento_perturbado_pos)

  # Recupera as POS Tagging do documento perturbado
  tokens_perturbado_index_palavra = []

  # Recupera os pos das sentenças
  pos_documento_perturbado = reg_documento_perturbado_pos['pos_documento']
  # print("pos_documento_perturbado:",pos_documento_perturbado)

  # Percorre as sentenças do documento
  for i, linha1 in enumerate(pos_documento_perturbado):
    # print("linha1:", linha1)

    # Percorre os tokens da sentença
    for j, linha2 in enumerate(linha1[0]):
      # print("linha2:", linha2)
      # Localiza o indice da palavra perturbada na sentença
      if token_predito == linha2:
        # Guarda o indice da palavra perturbada
        tokens_perturbado_index_palavra.append(j)

  # Verifica se encontrou o índice da palavra perturbada
  if len(tokens_perturbado_index_palavra) != 0:
      # Possui somente uma palavra perturbada
      if len(tokens_perturbado_index_palavra) == 1:
        return tokens_perturbado_index_palavra[0]
      else:
        return tokens_perturbado_index_palavra
  else:
    # Não encontrou o índice da palavra perturbada
    return -1

In [ ]:
def getIndicePerturbacao(_id_documento):

  id_documento_perturbado = ""

  # Verifica o tipo do documento
  #if int(_id_documento)/2 == 1:
  if "_pert_" in _id_documento:
    # Documento perturbado
    id_documento_perturbado = _id_documento

  else:
    # Pega o primeiro documento perturbado para localizar posição
    id_documento_perturbado = _id_documento + "_pert_0"
    #id_documento_perturbado = str(int(_id_documento) + 1)

  # Retorna o índice
  index_perturbacao = getIndicePalavraPerturbada(id_documento_perturbado)

  return index_perturbacao

## 5.2 Gera os arquivos para o Embedding Projector

### 5.2.1 Cria o diretório para os arquivos

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL + "projector"

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}".format(dirbase))
else:
    logging.info("Diretório já existe: {}".format(dirbase))

INFO:root:Diretório já existe: /content/SQUAD2_P/projector


### 5.2.2 Gera os embeddings dos documentos

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook

lista_embeddings = []
lista_embeddings_documento = []
lista_documentos = []
lista_documentos_tokenizado = []
lista_documentos_tokenizado_oov = []
lista_documentos_pos = []
lista_documentos_classe = []
lista_documentos_id = []
lista_documentos_origem = []

maior_sequencia = 0

total_tokens = 0

if CLASSE_DOCUMENTO != 2:
  documentos = lista_documentos_agrupados.loc[lista_documentos_agrupados['classe'] == CLASSE_DOCUMENTO]
else:
  documentos = lista_documentos_agrupados

# Barra de progresso dos documentos
documentos_bar = tqdm_notebook(documentos.iterrows(), desc=f"Documentos", unit=f" documento", total=len(documentos))

# Percorre os documentos
for i, linha_documento in documentos_bar:

    # Recupera o id do documento
    id_documento = linha_documento[0]
    # print("id_documento:",id_documento)
    # print("linha_documento['documento']:", linha_documento['documento'])

    # Recupera a classe documento (1-original 0-perturbado)
    classe = linha_documento['classe']
    #print("classe:",classe)

    # Localiza a POSTagging do documento agrupado
    lista_pos_documento = lista_documentos_agrupados_pos_indexado.loc[id_documento][0]
    # print("lista_pos_documento:",lista_pos_documento)
    # print("len(lista_pos_documento):",len(lista_pos_documento))

    # Troca o documento por uma versão da concatenação das palavras geradas pelo spaCy
    # Percorre a lista_pos concatenando a posição 0 dos tokens
    documento_concatenado = " ".join(concatenaListas(lista_pos_documento, pos=0))
    # print("documento_concatenado:", documento_concatenado)
    documento = documento_concatenado

    if CLASSE_DOCUMENTO != 1:
      # Recupera a posição do traço no id do arquivo
      traco_ix = id_documento.find("_")
      if traco_ix != -1:
        # Recupera o id da perturbacao até a posição do traço até o fim
        id_perturbacao = id_documento[:traco_ix]
      else:
        id_perturbacao = id_documento

    if POOLING_TOKENS == 0:

        # Recupera os embeddings
        if ESTRATEGIA_EMBEDDING == 1:
          # Gera embeddings da última camada do BERT
          token_embeddings, documento_tokenizado =  getEmbeddingsUltimaCamada(documento, model, tokenizer)
        else:
          # Gera embeddings concatenando as 4 últimas camadas do BERT
          token_embeddings, documento_tokenizado = getEmbeddingsConcat4UltimasCamadas(documento, model, tokenizer)

        # Guarda o maior tamanho de documento
        if len(documento_tokenizado) > maior_sequencia:
            maior_sequencia =  len(documento_tokenizado)

        # Guarda o total de tokens dos documentos
        total_tokens = total_tokens + len(documento_tokenizado)

        # Guarda os embeddings e o documento tokenizado
        lista_embeddings.append(token_embeddings)
        # Guarda os embeddings do documento consolidado pela média e removendo os tokens [CLS] e [SEP]
        lista_embeddings_documento.append(torch.mean(token_embeddings[1:-1], dim=0))
        lista_documentos.append(documento)
        lista_documentos_tokenizado.append(documento_tokenizado)
        lista_documentos_classe.append(classe)
        lista_documentos_id.append(id_documento)
        if CLASSE_DOCUMENTO != 1:
          lista_documentos_origem.append(id_perturbacao)

    else:
        # Recupera os embeddings
        if ESTRATEGIA_EMBEDDING == 1:
          # Gera embeddings da última camada do BERT
          token_embeddings, documento_tokenizado =  getEmbeddingsUltimaCamada(documento, model, tokenizer)
        else:
          # Gera embeddings concatenando as 4 últimas camadas do BERT
          token_embeddings, documento_tokenizado = getEmbeddingsConcat4UltimasCamadas(documento, model, tokenizer)

        # Combina os embeddings de palavras fora do vocabulário do BERT
        listaTokens, listaPOS, lista_tokens_OOV, listaEmbeddingsMEAN, listaEmbeddingsMAX =  getTokensEmbeddingsPOSSentenca(token_embeddings[1:-1],
                                                                                                        documento_tokenizado[1:-1],
                                                                                                        documento)

        # Guarda o maior tamanho de documento
        if len(listaTokens) > maior_sequencia:
            maior_sequencia =  len(listaTokens)

        # Guarda o total de tokens dos documentos
        total_tokens = total_tokens + len(listaTokens)

        # Guarda os embeddings e os os outros dados do documento
        lista_embeddings.append(listaEmbeddingsMEAN)
        # Guarda os embeddings do documento consolidado pela média e removendo os tokens [CLS] e [SEP]
        lista_embeddings_documento.append(torch.mean(token_embeddings[1:-1], dim=0))
        lista_documentos.append(documento)
        lista_documentos_tokenizado.append(listaTokens)
        lista_documentos_tokenizado_oov.append(lista_tokens_OOV)
        lista_documentos_pos.append(listaPOS)
        lista_documentos_classe.append(classe)
        lista_documentos_id.append(id_documento)
        if CLASSE_DOCUMENTO != 1:
          lista_documentos_origem.append(id_perturbacao)

Documentos:   0%|          | 0/21 [00:00<?, ? documento/s]

Mostra um documento processado.

In [ ]:
print(len(lista_embeddings[0]))
print(lista_documentos_tokenizado[0])
print(lista_documentos_classe[0])

8
['Como', 'os', 'bytes', 'de', 'bit', 'são', 'transmitidos', '?']
1


Quantidade de tokens nos documentos

In [ ]:
print("Quantidade de tokens:", total_tokens)

Quantidade de tokens: 168


Maior tamanho  de documento

In [ ]:
print("max_seq_length:", maior_sequencia)

max_seq_length: 8


### 5.2.3 Gera os arquivos para o Embedding Projector

Gera o sufixo do nome do arquivo

In [ ]:
def getSufixoNomeArquivo():

  sufixo_arquivo = "_"

  # Documento perturbados
  if CLASSE_DOCUMENTO == 0:
      sufixo_arquivo = sufixo_arquivo + "PERTDO" + "_P" + str(DOCUMENTOS_PERTURBADOS)
  else:
    # Documento originais
    if CLASSE_DOCUMENTO == 1:
      sufixo_arquivo = sufixo_arquivo + "DO"
    else:
      # Documento originais e perturbados
      if CLASSE_DOCUMENTO == 2:
        sufixo_arquivo = sufixo_arquivo + "DO_PERTDO"  + "_P" + str(DOCUMENTOS_PERTURBADOS) + "_CLASSE"

  # Sem pooling dos tokens
  if POOLING_TOKENS == 0:
    # Tamanho dos embeddings
    sufixo_arquivo = sufixo_arquivo + "_" + str(lista_embeddings[0].size()[1]) + TAMANHO_BERT

    # Não possui o prefixo pooling
  else:
    # Com pooling dos tokens
    if POOLING_TOKENS == 1:
      sufixo_arquivo = sufixo_arquivo + "_" + str(lista_embeddings[0][0].size()[0]) + TAMANHO_BERT

      # Adiciona o prefixo
      sufixo_arquivo = sufixo_arquivo + "_POOL"

  return sufixo_arquivo

Arquivos com os valores dos embeddings

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook
import csv

# Recupera o sufixo do nome do arquivo
sufixo_arquivo = getSufixoNomeArquivo()
#print("sufixo_arquivo:", sufixo_arquivo)

NOME_ARQUIVO_RECORD =  DIRETORIO_LOCAL + "projector/" + FILTRO_STR[IDDO] + "records_token_sentenca" + sufixo_arquivo + ".tsv"

# Abre o arquivo
with open(NOME_ARQUIVO_RECORD, 'w', encoding='utf8') as tsvfile:
  # Cria um arquivo separado por tab
    writer = csv.writer(tsvfile, delimiter='\t')

    # Barra de progresso dos embedings
    lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

    # Percorre os embeddings
    for i, documento_embedding in lista_embeddings_bar:

      if POOLING_TOKENS == 0:
        # Converte os tensores em numpy array
        documento_embedding_np =  documento_embedding.numpy()

        # Qtde de tokens do documento
        length = len(lista_documentos_tokenizado[i])

        # Escreve no arquivo os embeddings das palavras
        writer.writerows(documento_embedding_np[:length])

      else:
        # Converte os tensores em numpy array
        documento_embedding_np = []
        for linha in documento_embedding:
            novo = linha.numpy()
            documento_embedding_np.append(novo)

        # Qtde de tokens do documento
        length = len(lista_documentos_tokenizado[i])

        # Escreve no arquivo os embeddings das palavras
        writer.writerows(documento_embedding_np[:length])

      # Escreve no arquivo os embeddings do documento
      writer.writerows([lista_embeddings_documento[i].numpy()])

Embeddings:   0%|          | 0/21 [00:00<?, ? embedding/s]

Arquivo com os metadados dos embeddings

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook
import csv

# Recupera o sufixo do nome do arquivo
sufixo_arquivo = getSufixoNomeArquivo()
#print("sufixo_arquivo:", sufixo_arquivo)

NOME_ARQUIVO_META =  DIRETORIO_LOCAL + "projector/" + "DO" + FILTRO_DO[0] + "_meta_token_sentenca" + sufixo_arquivo + ".tsv"

# Abre o arquivo
with open(NOME_ARQUIVO_META, 'w', encoding='utf8') as tsvfile:
    # Define o escritor do arquivo
    writer = csv.writer(tsvfile, delimiter='\t')

    # Cabeçalho do arquivo
    # Sem pooling
    if POOLING_TOKENS == 0:

      # Sem classe
      if CLASSE_DOCUMENTO != 2:

        # Com o link da sequência de tokens da sentença
        if LIGACAO_PROXIMO_TOKEN == True:

          # Escreve o cabeçalho do arquivo
          writer.writerow(["Token", "Id", "Origem", "Index", "__next__", "Granularidade", "Tipo_Texto", "Sentença"])

          # Barra de progresso dos embedings
          lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

          # Contador da sequência
          conta_proximo = 1

          # Percorre os embeddings
          for i, documento_embedding in lista_embeddings_bar:

              # Qtde de tokens do documento
              length = len(lista_documentos_tokenizado[i])

              # Escreve a palavra e sua sentença
              for j in range(length):

                  # Transforma o conta_proximo que é o indicador da sequência em string
                  proximo = str(conta_proximo)

                  # Incrementa o contador da sequência
                  conta_proximo = conta_proximo + 1

                  # Monta o registro a ser salvo
                  s = [lista_documentos_tokenizado[i][j],
                       lista_documentos_id[i],
                       lista_documentos_origem[i],
                       str(j),
                       proximo,
                       "0",
                       lista_documentos_classe[i],
                       lista_documentos[i]]

                  # Escreve o registro no arquivo
                  writer.writerow(s)

              # Se chegou no último token da sequência coloca Branco para a próximo palavra
              proximo = ""

              # Escreve o rótulo do documento
              # Monta o registro a ser salvo
              s = [str(i),
                   lista_documentos_id[i],
                   lista_documentos_origem[i],
                   str(j),
                   proximo,
                   "1",
                   lista_documentos_classe[i]+2,
                   lista_documentos[i]]

              # Escreve o registro no arquivo
              writer.writerow(s)

        else:
          # Sem o link de ligação dos tokens da sentença
          # Escreve o cabeçalho do arquivo
          writer.writerow(["Token", "Id", "Origem", "Index", "Granularidade", "Tipo_Texto", "Sentença"])

          # Barra de progresso dos embedings
          lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

          # Percorre os embeddings
          for i, documento_embedding in lista_embeddings_bar:
              # Qtde de tokens do documento
              length = len(lista_documentos_tokenizado[i])

              # Escreve a palavra e sua sentença
              for j in range(length):

                # Monta o registro a ser salvo
                s = [lista_documentos_tokenizado[i][j],
                     lista_documentos_id[i],
                     lista_documentos_origem[i],
                     str(j),
                     "0",
                     lista_documentos_classe[i],
                     lista_documentos[i]]

                # Escreve o registro no arquivo
                writer.writerow(s)

          # Escreve o rótulo do documento
          # Monta o registro a ser salvo
          s = [str(i),
               lista_documentos_id[i],
               lista_documentos_origem[i],
               "-1",
               "1",
               lista_documentos_classe[i]+2,
               lista_documentos[i]]

          # Escreve o registro no arquivo
          writer.writerow(s)

      else:
        # Com classe

        # Com o link da sequência de tokens da sentença
        if LIGACAO_PROXIMO_TOKEN == True:

          # Escreve o cabeçalho do arquivo
          writer.writerow(["Token", "Id", "Origem", "Classe", "Perturbada", "Index", "__next__", "Granularidade", "Tipo_Texto", "Sentença"])

          # Barra de progresso dos embedings
          lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

          # Contador da sequência
          conta_proximo = 1

          # Percorre os embeddings
          for i, documento_embedding in lista_embeddings_bar:

              # Qtde de tokens do documento
              length = len(lista_documentos_tokenizado[i])

              # Procura o índice da palavra selecionada para perturbação no documento
              indice_palavra_perturbada = getIndicePerturbacao(lista_documentos_id[i])

              # Escreve a palavra e sua sentença
              for j in range(length):

                  # Transforma o conta_proximo que é o indicador da sequência em string
                  proximo = str(conta_proximo)

                  # Incrementa o contador da sequência
                  conta_proximo = conta_proximo + 1

                  # Identifica a posição da palavra selecionada para perturbação
                  perturbada = "0"
                  if indice_palavra_perturbada == j:
                    perturbada = "1"

                  # Monta o registro a ser salvo
                  s = [lista_documentos_tokenizado[i][j],
                       lista_documentos_id[i],
                       lista_documentos_origem[i],
                       lista_documentos_classe[i],
                       perturbada,
                       "-1",
                       proximo,
                       "0",
                       lista_documentos_classe[i],
                       lista_documentos[i]
                      ]

                  # Escreve o registro no arquivo
                  writer.writerow(s)

              # Escreve o rótulo do documento
              # Se chegou no último token da sequência coloca Branco para a próximo palavra
              proximo = ""

              # Incrementa o contador da sequência
              conta_proximo = conta_proximo + 1

              # Monta o registro a ser salvo
              s = [str(i),
                   lista_documentos_id[i],
                   lista_documentos_origem[i],
                   lista_documentos_classe[i],
                   "-1",
                   "-1",
                   proximo,
                   "1",
                   lista_documentos_classe[i]+2,
                   lista_documentos[i]
                   ]

              # Escreve o registro no arquivo
              writer.writerow(s)

        else:
          # Com Classe
          #Sem o link
          # Escreve o cabeçalho do arquivo
          writer.writerow(["Token", "Id", "Origem", "Classe", "Perturbada", "Index", "Granularidade", "Tipo_Texto", "Sentença"])

          # Barra de progresso dos embedings
          lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

          # Percorre os embeddings
          for i, documento_embedding in lista_embeddings_bar:

              # Qtde de tokens do documento
              length = len(lista_documentos_tokenizado[i])

              # Procura o índice da palavra selecionada para perturbação no documento
              indice_palavra_perturbada = getIndicePerturbacao(lista_documentos_id[i])

              # Escreve a palavra e sua sentença
              for j in range(length):

                # Identifica a posição da palavra selecionada para perturbação
                perturbada = "0"
                if indice_palavra_perturbada == j:
                  perturbada = "1"

                # Monta o registro a ser salvo
                s = [lista_documentos_tokenizado[i][j],
                     lista_documentos_id[i],
                     lista_documentos_origem[i],
                     lista_documentos_classe[i],
                     perturbada,
                     str(j),
                     "0",
                     lista_documentos_classe[i],
                     lista_documentos[i]]

                # Escreve o registro no arquivo
                writer.writerow(s)

              # Escreve o rótulo do documento
              # Monta o registro a ser salvo
              s = [str(i),
                   lista_documentos_id[i],
                   lista_documentos_origem[i],
                   lista_documentos_classe[i],
                   "-1",
                   "-1",
                   "1",
                   lista_documentos_classe[i]+2,
                   lista_documentos[i]]

              # Escreve o registro no arquivo
              writer.writerow(s)

    else:
      # Com polling
      # Sem classe
      if CLASSE_DOCUMENTO != 2:

        # Com o link da sequência de tokens da sentença
        if LIGACAO_PROXIMO_TOKEN == True:

          # Escreve o cabeçalho do arquivo
          writer.writerow(["Token", "POS-Tag", "OOV", "Id", "Origem", "Index", "__next__", "Granularidade", "Tipo_Texto", "Sentença"])

          # Barra de progresso dos embedings
          lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

          # Contador da sequência
          conta_proximo = 1

          # Percorre os embeddings
          for i, documento_embedding in lista_embeddings_bar:

              # Qtde de tokens do documento
              length = len(lista_documentos_tokenizado[i])

              # Escreve a palavra e sua sentença
              for j in range(length):

                # Transforma o conta_proximo que é o indicador da sequência em string
                proximo = str(conta_proximo)

                # Incrementa o contador da sequência
                conta_proximo = conta_proximo + 1

                # Monta o registro a ser salvo
                s = [lista_documentos_tokenizado[i][j],
                     lista_documentos_pos[i][j],
                     lista_documentos_tokenizado_oov[i][j],
                     lista_documentos_id[i],
                     lista_documentos_origem[i],
                     str(j),
                     proximo,
                     "0",
                     lista_documentos_classe[i],
                     lista_documentos[i]]

                # Escreve o registro no arquivo
                writer.writerow(s)

              # Escreve o rótulo do documento
              # Se chegou no último token da sequência coloca Branco para a próximo palavra
              proximo = ""

              # Incrementa o contador da sequência
              conta_proximo = conta_proximo + 1

              # Monta o registro a ser salvo
              s = [str(i),
                   lista_documentos_pos[i],
                   lista_documentos_tokenizado_oov[i],
                   lista_documentos_id[i],
                   lista_documentos_origem[i],
                   "-1",
                   proximo,
                   "1",
                   lista_documentos_classe[i]+2,
                   lista_documentos[i]]

              # Escreve o registro no arquivo
              writer.writerow(s)

        else:
            # Sem link de próximo

            # Escreve o cabeçalho do arquivo
            writer.writerow(["Token", "POS-Tag", "OOV", "Id", "Origem", "Index", "Granularidade", "Tipo_Texto", "Sentença"])

            # Barra de progresso dos embedings
            lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

            # Contador da sequência
            conta_proximo = 1

            # Percorre os embeddings
            for i, documento_embedding in lista_embeddings_bar:

                # Qtde de tokens do documento
                length = len(lista_documentos_tokenizado[i])

                # Escreve a palavra e sua sentença
                for j in range(length):

                  # Monta o registro a ser salvo
                  s = [lista_documentos_tokenizado[i][j],
                       lista_documentos_pos[i][j],
                       lista_documentos_tokenizado_oov[i][j],
                       lista_documentos_id[i],
                       lista_documentos_origem[i],
                       str(j),
                       "0",
                       lista_documentos_classe[i],
                       lista_documentos[i]
                       ]

                  # Escreve o registro no arquivo
                  writer.writerow(s)

            # Escreve o rótulo do documento
            # Monta o registro a ser salvo
            s = [str(i),
                 lista_documentos_pos[i],
                 lista_documentos_tokenizado_oov[i],
                 lista_documentos_id[i],
                 lista_documentos_origem[i],
                 str(i),
                 "1",
                 lista_documentos_classe[i]+2,
                 lista_documentos[i]
                 ]

            # Escreve o registro no arquivo
            writer.writerow(s)
      else:
        # Com classe

        # Com o link da sequência de tokens da sentença
        if LIGACAO_PROXIMO_TOKEN == True:

          # Escreve o cabeçalho do arquivo
          writer.writerow(["Token", "POS-Tag", "OOV", "Id", "Origem", "Classe", "Perturbada", "Index", "__next__", "Granularidade", "Tipo_Texto", "Sentença"])

          # Barra de progresso dos embedings
          lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

          # Contador da sequência
          conta_proximo = 1

          # Percorre os embeddings
          for i, documento_embedding in lista_embeddings_bar:

              # Qtde de tokens do documento
              length = len(lista_documentos_tokenizado[i])

              # Procura o índice da palavra selecionada para perturbação no documento
              indice_palavra_perturbada = getIndicePerturbacao(lista_documentos_id[i])

              # Escreve a palavra e sua sentença
              for j in range(length):

                # Transforma o conta_proximo que é o indicador da sequência em string
                proximo = str(conta_proximo)

                # Incrementa o contador da sequência
                conta_proximo = conta_proximo + 1

                # Identifica a posição da palavra selecionada para perturbação
                perturbada = "0"
                if indice_palavra_perturbada == j:
                  perturbada = "1"

                # Monta o registro a ser salvo
                s = [lista_documentos_tokenizado[i][j],
                     lista_documentos_pos[i][j],
                     lista_documentos_tokenizado_oov[i][j],
                     lista_documentos_id[i],
                     lista_documentos_origem[i],
                     lista_documentos_classe[i],
                     perturbada,
                     str(j),
                     proximo,
                     "0",
                     lista_documentos_classe[i],
                     lista_documentos[i]
                    ]

                # Escreve o registro no arquivo
                writer.writerow(s)

              # Escreve o rótulo do documento
              # Se chegou no último token da sequência coloca Branco para a próximo palavra
              proximo = ""

              # Incrementa o contador da sequência
              conta_proximo = conta_proximo + 1

              # Monta o registro a ser salvo
              s = [str(i),
                   lista_documentos_pos[i],
                   lista_documentos_tokenizado_oov[i],
                   lista_documentos_id[i],
                   lista_documentos_origem[i],
                   lista_documentos_classe[i],
                   "-1",
                   "-1",
                   proximo,
                   "1",
                   lista_documentos_classe[i]+2,
                   lista_documentos[i]
                  ]

              # Escreve o registro no arquivo
              writer.writerow(s)

        else:

          # Escreve o cabeçalho do arquivo
          writer.writerow(["Token", "POS-Tag", "OOV", "Id", "Origem", "Classe", "Perturbada", "Index", "Granularidade", "Tipo_Texto", "Sentença" ])

          # Barra de progresso dos embedings
          lista_embeddings_bar = tqdm_notebook(enumerate(lista_embeddings), desc=f"Embeddings", unit=f" embedding", total=len(lista_embeddings))

          # Percorre os embeddings
          for i, documento_embedding in lista_embeddings_bar:

              # Qtde de tokens do documento
              length = len(lista_documentos_tokenizado[i])

              # Procura o índice da palavra selecionada para perturbação no documento
              indice_palavra_perturbada = getIndicePerturbacao(lista_documentos_id[i])

              # Identifica a posição da palavra selecionada para perturbação
              perturbada = "0"
              if indice_palavra_perturbada == j:
                perturbada = "1"

              # Escreve a palavra e sua sentença
              for j in range(length):
                # Monta o registro a ser salvo
                s = [lista_documentos_tokenizado[i][j],
                     lista_documentos_pos[i][j],
                     lista_documentos_tokenizado_oov[i][j],
                     lista_documentos_id[i],
                     lista_documentos_origem[i],
                     lista_documentos_classe[i],
                     perturbada,
                     str(j),
                     "0",
                     lista_documentos_classe[i],
                     lista_documentos[i],
                    ]

                # Escreve o registro no arquivo
                writer.writerow(s)

              # Escreve o rótulo do documento
              # Monta o registro a ser salvo
              s = [str(i),
                   lista_documentos_pos[i],
                   lista_documentos_tokenizado_oov[i],
                   lista_documentos_id[i],
                   lista_documentos_origem[i],
                   lista_documentos_classe[i],
                   "-1",
                   "-1",
                   "1",
                   lista_documentos_classe[i]+2,
                   lista_documentos[i],
                   ]

              # Escreve o registro no arquivo
              writer.writerow(s)

Embeddings:   0%|          | 0/21 [00:00<?, ? embedding/s]

Faça o download dos arquivos **records_token_4096.tsv** e **meta_token_4096.tsv** e carregue em https://projector.tensorflow.org/ na opção load.

Faça o download dos arquivos gerados pelo notebook clicando na lateral esquerda no ícone "Arquivos".

Carrega os arquivos na ferramenta através do link "Load". Na opção existe um link botão para carregar o arquivo dos embeddings e um outro botão para carregar os metadados.

Você também pode utilizar um link a um arquivo de configuração config.json com a referência aos arquivos em algum repositório publico na internet, por exemplo github ou gist

Aqui um exemplo.

https://projector.tensorflow.org/?config=https://raw.githubusercontent.com/osmarbraz/cohebertv1projecao/main/config.json




### 5.3.4 Compacta e copia o arquivo do projetor para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

In [ ]:
# Nome do arquivo
NOME_ARQUIVO_PROJECTOR_COMPACTADO = "projector.zip"

Compacta os arquivos.

Usa o zip para compactar:
*   `-r` Compacta o diretório
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens


In [ ]:
!zip -r -o -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PROJECTOR_COMPACTADO" "$DIRETORIO_LOCAL""/projector/"

Copia o arquivo compactado para o GoogleDrive



In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
    # Copia o arquivo original
    # !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_PROJECTOR_COMPACTADO" "$DIRETORIO_DRIVE"

    logging.info("Terminei a cópia")

INFO:root:Terminei a cópia


## 5.3 Projeção dos embeddings

### Configuração

Verifica a versão do tensorflow

In [ ]:
try:
  # %tensorflow_version só existe no Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Importa a biblioteca

In [ ]:
# Importa de biblioteca
from tensorboard.plugins import projector

### Configura o diretório dos logs e arquivos de configuração




In [ ]:
# Configure um diretório de logs
log_dir ="/content/projector/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

### Cria os arquivos de configuração dos embeddings

In [ ]:
# Configura o projetor
config = projector.ProjectorConfig()

# Configuração do primeiro conjunto de embeddings sem pooling
embedding = config.embeddings.add()
# Nome do tensor
embedding.tensor_name = "Cohebert: concat 4 últimas camadas pool BERTimbau large"
# Caminho para os metadados
embedding.metadata_path = NOME_ARQUIVO_META
# Caminho para os tensores
embedding.tensor_path = NOME_ARQUIVO_RECORD
# Salva o arquivo de configuração
projector.visualize_embeddings(log_dir, config)

### Mata o processo

Se executar novamente o notebook é necessário matar o processo do tensorprojector.

In [ ]:
# Mata o processo do tensorboard
#!kill 407

### Visualizando a projeção

Na caixa de seleção selecione "PROJECTOR" no lugar de "INACTIVE"

In [ ]:
# Agora execute o tensorboard nos dados de log que acabamos de salvar.
# %tensorboard --logdir /content/projector